can run locally

In [1]:
!pip install torch torchvision numpy tqdm matplotlib plotnine pandas


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# train

In [ ]:
import copy
import os
from plotnine import *
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
from tqdm import tqdm
from utils import *


device = get_device(disable_mps=False)
set_env()


#
# data
#


data_path = get_current_dir().parent / "data"
dataset_path = get_current_dir().parent / "datasets"
weights_path = get_current_dir().parent / "weights"

os.makedirs(data_path, exist_ok=True)
os.makedirs(dataset_path, exist_ok=True)
os.makedirs(weights_path, exist_ok=True)


dataset = "cifar100"

if dataset == "cifar10":
    num_classes = 10
    trainset = torchvision.datasets.CIFAR10(root=dataset_path, train=True, download=True, transform=torchvision.transforms.ToTensor())
    testset = torchvision.datasets.CIFAR10(root=dataset_path, train=False, download=True, transform=torchvision.transforms.ToTensor())

    original_images_train_np = np.array(trainset.data)
    original_labels_train_np = np.array(trainset.targets)

    original_images_test_np = np.array(testset.data)
    original_labels_test_np = np.array(testset.targets)

elif dataset == "cifar100":
    num_classes = 100
    trainset = torchvision.datasets.CIFAR100(root=dataset_path, train=True, download=True, transform=torchvision.transforms.ToTensor())
    testset = torchvision.datasets.CIFAR100(root=dataset_path, train=False, download=True, transform=torchvision.transforms.ToTensor())

    original_images_train_np = np.array(trainset.data)
    original_labels_train_np = np.array(trainset.targets)

    original_images_test_np = np.array(testset.data)
    original_labels_test_np = np.array(testset.targets)

elif dataset == "imagenette":
    num_classes = 10
    imgpath = dataset_path / "imagenette2"
    transform = torchvision.transforms.Compose(
        [
            torchvision.transforms.Resize((224, 224)),
            torchvision.transforms.ToTensor(),
        ]
    )
    trainset = torchvision.datasets.Imagenette(root=dataset_path, split="train", download=(not (imgpath / "train").exists()), transform=transform)
    testset = torchvision.datasets.Imagenette(root=dataset_path, split="val", download=(not (imgpath / "val").exists()), transform=transform)

    # preallocate
    num_train = len(trainset)
    num_test = len(testset)
    original_images_train_np = np.empty((num_train, 224, 224, 3), dtype=np.uint8)
    original_labels_train_np = np.empty(num_train, dtype=np.int64)
    original_images_test_np = np.empty((num_test, 224, 224, 3), dtype=np.uint8)
    original_labels_test_np = np.empty(num_test, dtype=np.int64)

    for idx, (image, label) in tqdm(enumerate(trainset), total=num_train, desc="preprocessing trainset", ncols=100):
        original_images_train_np[idx] = (np.transpose(image.numpy(), (1, 2, 0)) * 255).astype(np.uint8)
        original_labels_train_np[idx] = label

    for idx, (image, label) in tqdm(enumerate(testset), total=num_test, desc="preprocessing testset", ncols=100):
        original_images_test_np[idx] = (np.transpose(image.numpy(), (1, 2, 0)) * 255).astype(np.uint8)
        original_labels_test_np[idx] = label

else:
    assert False

images_train_np = original_images_train_np / 255.0  # map to [0, 1]
images_test_np = original_images_test_np / 255.0
labels_train_np = original_labels_train_np
labels_test_np = original_labels_test_np


#
# tuning model
#


from torchvision.models import resnet152, ResNet152_Weights

model = resnet152(weights=ResNet152_Weights.IMAGENET1K_V2)
model.fc = nn.Linear(2048, num_classes)

model = model.to(device)
model.train()


# 
# eval
# 


def add_overlay(background: Image.Image, overlay: Image.Image, opacity: int) -> Image.Image:
    # opacity range: 0 (transparent) to 255 (opaque)
    overlay = overlay.resize(background.size)
    result = Image.new("RGBA", background.size)
    result.paste(background, (0, 0))
    mask = Image.new("L", overlay.size, opacity)
    result.paste(overlay, (0, 0), mask)
    return result


test_mods = []
free_mem()

images_test_np_copy = images_test_np.copy()
labels_test_np_copy = labels_test_np.copy()

if True:
    args = {
        "opacity": 12,  # range: 0 (transparent) to 255 (opaque)
    }
    test_mods.append({"hcaptcha_mask": args})
    maskpath = get_current_dir().parent / "data" / "masks" / "mask.png"
    mask = Image.open(maskpath)
    for i in range(len(images_test_np_copy)):
        images_test_np_copy[i] = add_overlay(Image.fromarray((255 * images_test_np_copy[i]).astype(np.uint8)), mask, args["opacity"]).convert("RGB")

# eval

Files already downloaded and verified
Files already downloaded and verified
